## 实现VGG系列

vgg网络的结构比较简单，就是简单的网络堆叠。其中有比较经典的VGG11，VGG13，VGG16，VGG19

In [4]:
#VGG11/13/19 in PyTorch”“”
import torch
import torch.nn as nn

In [86]:
cfg={
    'VGG11':[64,'M',128,'M',256,256,'M',512,512,'M',512,512,'M'],
    'VGG13':[64,64,'M',128,128,'M',256,256,'M',512,512,'M',512,512,'M'],
    'VGG16':[64,64,'M',128,128,'M',256,256,256,'M',512,512,512,'M',512,512,512,'M'],
    'VGG19':[64,64,'M',128,128,'M',256,256,256,256,'M',512,512,512,512,'M',512,512,512,512,'M'],
}

In [123]:
class VGG(nn.Module):
    def __init__(self,vgg_name):
        super(VGG,self).__init__()
        self.features=self._make_layers(cfg[vgg_name])
        self.avgpool=nn.AdaptiveAvgPool2d((7,7))
        self.classifier=nn.Sequential(
            nn.Linear(512*7*7,4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096,4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096,2),)
    def forward(self,x):
        out=self.features(x)
        out=self.avgpool(out)
        out=out.view(out.size(0),-1)
        out=self.classifier(out)
        return out
    def _make_layers(self,cfg):
        layers=[]
        in_channels=3
        for x in cfg:
            if x=='M':#如果是M就添加池化层
                layers+=[nn.MaxPool2d(kernel_size=2,stride=2)]
            else:
                layers+=[nn.Conv2d(in_channels,x,kernel_size=3,padding=1),
                        nn.BatchNorm2d(x),
                        nn.ReLU(inplace=True)]
                in_channels=x
        layers+=[nn.AvgPool2d(kernel_size=1,stride=1)]
        return nn.Sequential(*layers)

In [124]:
def test():
    model=VGG('VGG16')
    x=torch.randn(2,3,224,224)
    y=model(x)
    print(y.size())

In [125]:
test()

torch.Size([2, 2])
